# Code for Running Experiments

In [9]:
# from datasets.datasets import mvtec
import os
import numpy as np
from utils.general_utils import create_mvtec_supervised
from datasets.datasets import mvtec

### Creating MVTEC supervised dataset
# First, we get statistics where we find what is the distribution of the number of images per anomaly-type, across all categories to
# see how many average number of images in each of the test anomalous classes
num_vals = []
path = 'datasets/mvtec_anomaly_detection/supervised/'
for cat in os.listdir(path):
    if "." not in cat:
        for anom_class in os.listdir(path+cat+"/test/"):
            if "." not in anom_class:
                if anom_class != "good":
                    num_vals.append(len(os.listdir(path+cat+"/test/"+anom_class)))

num_vals = np.array(num_vals)
print(num_vals.min(), num_vals.max(), num_vals.mean(), num_vals.std(), sep="\n")
# On average, we have about 17 images per anomalous class. In practice, we don't often have access to that many anomalous cases, so let's use 20% for testing and 80% for training.

create_supervised = False # we will set this to false after it's done, but we want to keep track of the code
if create_supervised: create_mvtec_supervised()

6
24
13.863013698630137
4.457762410912128


In [6]:
mvtec_bottle_train = mvtec(train=True, task='bottle', unsupervised=False)
mvtec_bottle_train[0]

datasets/mvtec_anomaly_detection/supervised/bottle/ground_truth/broken_small/001_mask.png


{'image': tensor([[[255, 255, 255,  ..., 255, 255, 255],
          [255, 255, 255,  ..., 255, 255, 255],
          [255, 255, 255,  ..., 255, 255, 255],
          ...,
          [255, 255, 255,  ..., 255, 255, 255],
          [255, 255, 255,  ..., 255, 255, 255],
          [255, 255, 255,  ..., 255, 255, 255]],
 
         [[255, 255, 255,  ..., 255, 255, 255],
          [255, 255, 255,  ..., 255, 255, 255],
          [255, 255, 255,  ..., 255, 255, 255],
          ...,
          [255, 255, 255,  ..., 255, 255, 255],
          [255, 255, 255,  ..., 255, 255, 255],
          [255, 255, 255,  ..., 255, 255, 255]],
 
         [[255, 255, 255,  ..., 255, 255, 255],
          [255, 255, 255,  ..., 255, 255, 255],
          [255, 255, 255,  ..., 255, 255, 255],
          ...,
          [255, 255, 255,  ..., 255, 255, 255],
          [255, 255, 255,  ..., 255, 255, 255],
          [255, 255, 255,  ..., 255, 255, 255]]], dtype=torch.uint8),
 'label': 1,
 'ground_truth': tensor([[[0, 0, 0,  ...,

In [8]:
mvtec_bottle_train.filenames[0]

'datasets/mvtec_anomaly_detection/supervised/bottle/train/broken_small/001.png'